In [10]:
# Lyonia UV/Vis Parser and Merger

import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt

# Set your working directory to the folder with your files
DATA_DIR = Path("~/Documents/ScienceProjects/2025/LyoniaUV").expanduser()
TXT_DIR = DATA_DIR  # assuming all .txt files are in this folder
SAMPLE_INFO_FILE = DATA_DIR / "UVTrans_Lyonia_13May2025_JW.csv"  # update if name differs


In [11]:
# --- Step 1: Parse each UV/Vis text file ---
def parse_txt_file(file_path):
    with open(file_path, 'r') as f:
        lines = f.readlines()

    # Find where spectral data begins
    start_idx = next(i for i, line in enumerate(lines) if '>>>>>Begin Spectral Data<<<<<' in line) + 1
    spectral_data = []

    for line in lines[start_idx:]:
        if line.strip() == "":
            continue
        try:
            wavelength, value = map(float, line.strip().split())
            spectral_data.append((wavelength, value))
        except ValueError:
            continue  # skip bad lines

    df = pd.DataFrame(spectral_data, columns=['Wavelength', 'Value'])
    df['Filename'] = file_path.name
    return df


In [12]:
# --- Step 2: Load metadata spreadsheet ---
metadata = pd.read_csv(SAMPLE_INFO_FILE)


In [13]:
# --- Step 3: Fill missing prefixes (for long-suffix files) ---
metadata['File_prefix'] = metadata['File_prefix'].fillna("WindowProjectSample")


In [14]:
# --- Step 4: Build 'FilenameBase' column ---
def build_filename_base(row):
    try:
        num = int(float(str(row['filename']).split('_')[0]))
    except:
        return None
    prefix = row['File_prefix']
    return f"{prefix}_Transmission__{num}__"

metadata['FilenameBase'] = metadata.apply(build_filename_base, axis=1)

# Create a lookup dict for exact or prefix match
filename_lookup = {fb: md_row for fb, md_row in metadata.set_index('FilenameBase').iterrows() if fb}


In [15]:
# --- Step 5: Parse all txt files ---
txt_files = list(TXT_DIR.glob("*.txt"))
spectra = pd.concat([parse_txt_file(f) for f in txt_files], ignore_index=True)


In [16]:
# --- Step 6: Match metadata ---
def match_metadata(filename):
    if filename in filename_lookup:
        return filename_lookup[filename]
    for base, row in filename_lookup.items():
        if base and base in filename:
            return row
    return pd.Series([None]*len(metadata.columns), index=metadata.columns)

# Apply matching
meta_cols = metadata.columns.tolist()
spectra_with_meta = spectra.copy()
spectra_with_meta[meta_cols] = spectra_with_meta['Filename'].apply(match_metadata)

# Drop rows that didn't match metadata
merged = spectra_with_meta.dropna(subset=['plant', 'species'])


In [17]:
# --- Step 7: Save merged data ---
merged.to_csv(DATA_DIR / "merged_uvvis_data.csv", index=False)


In [18]:
# Identify all unmatched files (no metadata joined)
unmatched = spectra_with_meta[spectra_with_meta['plant'].isna()]

# Show just the unique filenames
unmatched_files = unmatched['Filename'].unique()

# Print them
print("Unmatched files:")
for fname in unmatched_files:
    print(fname)

# Optional: How many
print(f"\nTotal unmatched files: {len(unmatched_files)}")

Unmatched files:
WindowProjectSample3_Transmission__52__00052.txt

Total unmatched files: 1
